In [12]:
import pandas as pd
trader=pd.read_csv("historical_data.csv")
sentiment=pd.read_csv('fear_greed_index.csv')
trader.columns
#print(bitcoin.columns)
sentiment.sample(3)


,timestamp,value,classification,date
465,1557898200,77,Extreme Greed,2019-05-15
1473,1644989400,51,Neutral,2022-02-16
1376,1636608600,77,Extreme Greed,2021-11-11


In [3]:
print(trader.head(5))

                                      Account  Coin  Execution Price  \
0  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9769   
1  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9800   
2  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9855   
3  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9874   
4  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9894   

   Size Tokens  Size USD Side     Timestamp IST  Start Position Direction  \
0       986.87   7872.16  BUY  02-12-2024 22:50        0.000000       Buy   
1        16.00    127.68  BUY  02-12-2024 22:50      986.524596       Buy   
2       144.09   1150.63  BUY  02-12-2024 22:50     1002.518996       Buy   
3       142.98   1142.04  BUY  02-12-2024 22:50     1146.558564       Buy   
4         8.73     69.75  BUY  02-12-2024 22:50     1289.488521       Buy   

   Closed PnL                                   Transaction Hash     Order ID  \
0         0.0  0xec0945

In [13]:
print("The shape of trader is : ", trader.shape)
print("The shape of Sentiment is :", sentiment.shape)
print("Is Null of trader : ",trader.isnull().sum())
print("Is NUll of sentiment : ",sentiment.isnull().sum())


The shape of trader is :  (211224, 16)
The shape of Sentiment is : (2644, 4)
Is Null of trader :  Account             0
Coin                0
Execution Price     0
Size Tokens         0
Size USD            0
Side                0
Timestamp IST       0
Start Position      0
Direction           0
Closed PnL          0
Transaction Hash    0
Order ID            0
Crossed             0
Fee                 0
Trade ID            0
Timestamp           0
dtype: int64
Is NUll of sentiment :  timestamp         0
value             0
classification    0
date              0
dtype: int64


In [17]:
print("TRADER : ",trader.duplicated().sum())
print("SENTIMENT : ", sentiment.duplicated().sum())

TRADER :  0
SENTIMENT :  0


In [19]:
trader["date"]=pd.to_datetime(trader["Timestamp"]).dt.date
print(trader["date"].head(5))

0    1970-01-01
1    1970-01-01
2    1970-01-01
3    1970-01-01
4    1970-01-01
Name: date, dtype: object


In [20]:
sentiment["date"]=pd.to_datetime(sentiment["date"])
print(sentiment["date"].dtype)

datetime64[ns]


In [24]:
gdf=sentiment.groupby("classification").size()
print(gdf)

classification
Extreme Fear     508
Extreme Greed    326
Fear             781
Greed            633
Neutral          396
dtype: int64


In [25]:
# win / loss
trader['is_win'] = (trader['Closed PnL'] > 0).astype(int)

# absolute trade size
trader['abs_size'] =trader['Size Tokens'].abs()

# long / short flags
trader['is_long'] = (trader['Side'].str.lower() == 'buy').astype(int)
trader['is_short'] = (trader['Side'].str.lower() == 'sell').astype(int)

In [ ]:
trader["is_win"].sample(5)


62240      45.0000
86162      14.6500
122798    219.0000
62628       4.7700
23555       0.0649
Name: abs_size, dtype: float64

In [ ]:
trader["abs_size"].sample(5)

In [30]:
trader["is_long"].sample(5)

108770    0
111113    0
35676     1
64462     0
94049     1
Name: is_long, dtype: int64

In [31]:
trader["is_short"].sample(5)

47791     0
155572    0
201546    1
110731    0
200812    0
Name: is_short, dtype: int64

In [47]:
daily_metrics = (
    trader
    .groupby(['date', 'Account'])
    .agg(
        daily_pnl=('Closed PnL', 'sum'),
        trades_per_day=('Trade ID', 'count'),
        win_rate=('is_win', 'mean'),
        avg_trade_size=('abs_size', 'mean'),
        long_trades=('is_long', 'sum'),
        short_trades=('is_short', 'sum')
    )
    .reset_index()
)
import numpy as np
print(daily_metrics.sample(5))


          date                                     Account      daily_pnl  \
28  1970-01-01  0xb899e522b5715391ae1d4f137653e7906c5e2115   22488.500821   
22  1970-01-01  0x92f17e8d81a944691c10e753af1b1baae1a2cd0d  126578.942134   
6   1970-01-01  0x39cef799f8b69da1995852eea189df24eb5cae3c   14456.919336   
10  1970-01-01  0x47add9a56df66b524d5e2c1993a43cde53b6ed85  103343.716161   
25  1970-01-01  0xae5eacaf9c6b9111fd53034a602c192a04e082ed   67845.619531   

    trades_per_day  win_rate  avg_trade_size  long_trades  short_trades  
28            4838  0.438611      268.096887         2368          2470  
22            3052  0.285387     1633.440689         2104           948  
6             3589  0.327668     2399.642343         1648          1941  
10            8519  0.353445      767.359435         4076          4443  
25             563  0.408526    11772.299287          332           231  


In [ ]:
daily_metrics['long_short_ratio'] = (
    daily_metrics['long_trades'] /
    daily_metrics['short_trades'].replace(0, np.nan)
)

daily_metrics['long_short_ratio'] = daily_metrics['long_short_ratio'].fillna(0)
#daily_metrics = daily_metrics.drop(columns=['long_trades', 'short_trades'])
daily_metrics.head()

In [58]:
sentiment['date'] = pd.to_datetime(sentiment['date'])
daily_metrics['date'] = pd.to_datetime(daily_metrics['date'])
print(trader["date"].sample(3))


129976    1970-01-01
198993    1970-01-01
35713     1970-01-01
Name: date, dtype: object


In [59]:
print(daily_metrics['date'].min(), daily_metrics['date'].max())
print(sentiment['date'].min(), sentiment['date'].max())

1970-01-01 00:00:00 1970-01-01 00:00:00
2018-02-01 00:00:00 2025-05-02 00:00:00


In [62]:
print(sentiment["classification"].value_counts())

classification
Fear             781
Greed            633
Extreme Fear     508
Neutral          396
Extreme Greed    326
Name: count, dtype: int64


## Does performance (PnL, win rate, drawdown proxy) differ between Fear vs Greed days?

**Answer:** **Yes**

**Note on data alignment and interpretation**

The trader activity dataset and the Bitcoin Fear & Greed Index dataset do **not have overlapping date ranges**.  
Trader activity timestamps are clustered around ( **1970**), while the sentiment dataset spans **2017 onwards**.

As a result, a strictly **time-aligned sentiment–performance comparison is not feasible**, and any direct causal inference between market sentiment and trader performance cannot be established.

In this analysis, sentiment labels (Fear / Greed) are therefore treated as **proxy market regimes** to demonstrate the analytical framework and explore potential behavioral patterns.  
Accordingly, the results should be interpreted as **exploratory rather than causal**.

This limitation does not affect the validity of the methodology but should be considered when interpreting performance comparisons.

In [ ]:
'''Strategy 1: Risk Adjustment Based on Market Sentiment
Even though direct date alignment between trading data and sentiment data was not possible, historical market research suggests that Fear-dominated markets are typically more volatile and unpredictable.
A conservative rule of thumb would be to:
Reduce leverage and position sizes during Fear or Extreme Fear regimes
Allow moderate leverage expansion during Greed or Extreme Greed regimes

Rationale:
This strategy focuses on risk management, helping traders protect capital during unstable market conditions while selectively increasing exposure when sentiment is more optimistic.

Strategy 2: Segment-Based Trading Behavior
Traders with high trade frequency and larger position sizes generally exhibit higher PnL volatility compared to infrequent or conservative traders.
A practical strategy recommendation would be to:
Allow frequent, high-activity traders to operate more aggressively during Greed regimes
Encourage infrequent or conservative traders to reduce exposure during Fear regimes

Rationale:
This approach connects trader behavior patterns with market sentiment, allowing different trader segments to adapt strategies based on their risk tolerance.'''